<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/Viste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Paso 1: Instalación de paquetes necesarios
print("Instalando paquetes necesarios...")

try:
    import torch
    import torchvision
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    print("Paquetes ya instalados.")
except ImportError as e:
    print(f"Error al importar paquetes: {e}")
    print("Instalando paquetes faltantes...")
    !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    !pip install opencv-python
    import torch
    import torchvision
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    print("Paquetes instalados correctamente.")

Instalando paquetes necesarios...
Paquetes ya instalados.


In [2]:
# Paso 2: Importación de bibliotecas y configuración del dispositivo
print("Importando bibliotecas y configurando el dispositivo...")

import torch
import torchvision
from torchvision import transforms
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando el dispositivo: {device}")

Importando bibliotecas y configurando el dispositivo...
Usando el dispositivo: cpu


In [3]:
# Paso 3: Carga de modelos pre-entrenados
print("Cargando modelos pre-entrenados...")

# Modelo de detección de personas (Faster R-CNN pre-entrenado en COCO)
person_detection_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
person_detection_model.to(device)
person_detection_model.eval()

# Modelo de reconocimiento de actividades (ResNet3D pre-entrenado en Kinetics-400)
activity_recognition_model = torchvision.models.video.r3d_18(pretrained=True)
activity_recognition_model.to(device)
activity_recognition_model.eval()

print("Modelos cargados correctamente.")

Cargando modelos pre-entrenados...


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 121MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None

Modelos cargados correctamente.


In [8]:
# Paso 4: Definición de transformaciones y funciones auxiliares
print("Definiendo transformaciones y funciones auxiliares...")

# Transformación para el modelo de detección
detection_transform = transforms.Compose([
    transforms.ToTensor()
])

# Transformación para el modelo de actividad
activity_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.43216, 0.394666, 0.37645],
        std=[0.22803, 0.22145, 0.216989]),
])

# Función para obtener etiquetas de las clases de Kinetics-400
def get_kinetics_class_labels():
    import urllib.request
    url = "https://raw.githubusercontent.com/jeffreyhuang1/kinetics-400-classnames/master/kinetics_classnames.txt"
    class_names = []
    with urllib.request.urlopen(url) as f:
        for line in f.readlines():
            class_name = line.decode('utf-8').rstrip()
            if class_name:
                class_names.append(class_name)
    return class_names

kinetics_class_names = get_kinetics_class_labels()

print("Transformaciones y funciones definidas correctamente.")

Definiendo transformaciones y funciones auxiliares...


HTTPError: HTTP Error 404: Not Found

In [5]:
# Paso 5: Procesamiento del video
print("Procesando el video...")

# Ruta del video de entrada
video_path = "input_video.mp4"  # Reemplaza con la ruta de tu video

# Inicializamos el lector de video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error al abrir el video.")
else:
    print("Video abierto correctamente.")

frame_rate = cap.get(cv2.CAP_PROP_FPS)
print(f"Frame rate del video: {frame_rate} FPS")

# Variables para almacenar datos
activity_sequences = []
current_sequence = []
sequence_start_time = 0
activity_durations = {}

frame_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Fin del video.")
            break

        frame_count += 1
        print(f"Procesando frame {frame_count}")

        # Convertimos el frame a imagen PIL y aplicamos la transformación
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        input_tensor = detection_transform(pil_image).to(device)

        # Detección de personas
        with torch.no_grad():
            detections = person_detection_model([input_tensor])[0]

        # Filtramos detecciones para mantener solo personas con alta confianza
        person_boxes = []
        for idx, label in enumerate(detections['labels']):
            if label == 1 and detections['scores'][idx] > 0.8:
                person_boxes.append(detections['boxes'][idx].cpu().numpy())

        # Si se detecta al menos una persona
        if person_boxes:
            # Añadimos el frame actual a la secuencia
            current_sequence.append(frame)
            if len(current_sequence) == 16:
                # Tenemos una secuencia de 16 frames, suficiente para el modelo
                # Preprocesamos la secuencia
                clip = []
                for img in current_sequence:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = Image.fromarray(img)
                    img = activity_transform(img)
                    clip.append(img)

                # Convertimos la secuencia en tensor
                clip = torch.stack(clip).permute(1, 0, 2, 3).unsqueeze(0).to(device)

                # Reconocimiento de actividad
                with torch.no_grad():
                    outputs = activity_recognition_model(clip)
                    probabilities = torch.nn.functional.softmax(outputs, dim=1)
                    top5_prob, top5_catid = torch.topk(probabilities, 5)

                # Obtenemos la actividad con mayor probabilidad
                activity = kinetics_class_names[top5_catid[0][0]]
                print(f"Actividad detectada: {activity}")

                # Registramos la duración de la actividad
                if activity in activity_durations:
                    activity_durations[activity] += (16 / frame_rate)
                else:
                    activity_durations[activity] = (16 / frame_rate)

                # Reiniciamos la secuencia actual
                current_sequence = []
        else:
            # Si no se detecta persona, reiniciamos la secuencia
            current_sequence = []

except Exception as e:
    print(f"Ocurrió un error durante el procesamiento: {e}")

finally:
    cap.release()
    print("Liberando recursos del video.")

print("Procesamiento completado.")

Procesando el video...
Error al abrir el video.
Frame rate del video: 0.0 FPS
Fin del video.
Liberando recursos del video.
Procesamiento completado.


In [6]:
# Paso 6: Mostrar resultados
print("Resultados:")
for activity, duration in activity_durations.items():
    print(f"Actividad: {activity}, Duración: {duration:.2f} segundos")

Resultados:
